# Nonlinear minimization problems (Unit 3.6)

We consider problems of the form
$$\text{find } u \in V \text{ s.t. } E(u) \leq E(v)\ \ \forall  v \in V.$$

In [1]:
import netgen.gui
%gui tk
import tkinter
from ngsolve import *
import ngsolve.internal as ngint
ngint.viewoptions.drawedges = 1


Scalar minimization problems

As a first example we take $V=H_0^1$ and
$$E(u)=\int_\Omega |\nabla u|^2 + u^4−fu dx.$$
The minimization is equivalent to solving the nonlinear PDE:
$$−\Delta u+4u^3=f \text{ in }\Omega$$
We solve the PDE with a Newton iteration.

In [2]:
from mesh_util import uniform_1d_mesh

mesh = Mesh (uniform_1d_mesh())
V = H1(mesh, order=4, dirichlet='left|right')
u = V.TrialFunction()


To solve the problem we use the "SymbolicEnergy" integrator. Based on the symbolic description of the energy functional, it is able to

- evaluate the energy functional ("Energy")
$$ E(u)\hspace{2em}(E:V\rightarrow \mathbb{R})$$

- compute the Gateau derivative for a given $u$ ("Apply"):
$$A(u)(v)=E'(u)(v)\hspace{2em} (A(u):V\rightarrow \mathbb{R})$$
- compute the second derivative ("AssembleLinearization")
$$(\delta A)(w)(u,v)\hspace{2em}(\delta A(w):V\times V\rightarrow \mathbb{R})$$

In [3]:
a = BilinearForm (V, symmetric=False)
a += SymbolicEnergy ( grad(u)*grad(u) + u*u*u*u-u )


We recall the Newton iteration (cf. unit-3.5 ) we make the loop:
- Given an initial guess $u_0$ 
- loop over $i=0,\dots$ until convergence:

   - Compute linearization: $A u^i+\delta A(u^i)\Delta u^i=0$:

      - $f^i=Au^i$

      - $B^i=\delta A(u^i)$
      - Solve $B^i\Delta u^i=−f^i$

    - Update $u^{i+1}=u^i+\Delta u^i$

    - Evaluate stopping criteria

- Evaluate $E(u^{i+1})

As a stopping criteria we take $\langle Au^i,\Delta u^i\rangle=\langle Au^i,Au^i\rangle_{(B^i)^{−1}}<\epsilon$
.

In [4]:
def SolveNonlinearMinProblem(a,gfu):
    res = gfu.vec.CreateVector()
    du  = gfu.vec.CreateVector()

    for it in range(12):
        print ("Newton iteration", it)
        print ("energy = ", a.Energy(gfu.vec))
    
        #solve linearized problem:
        a.Apply (gfu.vec, res)
        a.AssembleLinearization (gfu.vec)
        inv = a.mat.Inverse(V.FreeDofs())
        du.data = inv * res
    

        #update iteration
        gfu.vec.data -= du

        #stopping criteria
        stopcritval = sqrt(abs(InnerProduct(du,res)))
        print ("<A u",it,", A u",it,">_{-1}^0.5 = ", stopcritval)
        if stopcritval < 1e-13:
            break
        Redraw(blocking=True)

In [5]:
gfu = GridFunction (V)
gfu.vec[:] = 0
Draw(gfu,mesh,"u")

SolveNonlinearMinProblem(a,gfu)

print ("energy = ", a.Energy(gfu.vec))
    

Newton iteration 0
energy =  0.0
<A u 0 , A u 0 >_{-1}^0.5 =  0.2041241452319308
Newton iteration 1
energy =  -0.02082713293650777
<A u 1 , A u 1 >_{-1}^0.5 =  0.0001240912213075285
Newton iteration 2
energy =  -0.020827140635829355
<A u 2 , A u 2 >_{-1}^0.5 =  1.4453148544145113e-10
Newton iteration 3
energy =  -0.02082714063582936
<A u 3 , A u 3 >_{-1}^0.5 =  7.760472570283761e-17
energy =  -0.020827140635829355


Nonlinear elasticity

We consider a beam which is fixed on one side and is subject to gravity only. We assume a Neo-Hookean hyperelastic material. The model is a nonlinear minimization problem with
$$E(v):=\int_\Omega \mu^2 (\text{tr}(F^T F - I) + \frac{2 \mu}{\lambda} \text{det}(F^TF)^{\frac{\lambda}{ 2 \mu}} - 1) - \gamma (f,v) dx$$
where $\mu$ and $\lambda$ are the Lamé parameters and $F=I+Du$ where $v:\Omega \rightarrow \mathbb{R}^2$ is the sought for displacement.

In [6]:

# E module and poisson number:
E, nu = 210, 0.2
# Lamé constants:
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

V = H1(mesh, order=2, dirichlet="left", dim=mesh.dim)
u  = V.TrialFunction()

#gravity:
force = CoefficientFunction( -1 )

# some utils:
def IdentityCF(dim):
    return CoefficientFunction( tuple( [1 if i==j else 0 for i in range(dim) for j in range(dim)]), dims=(dim,dim) )

def Trace(mat):
    return sum( [mat[i,i] for i in range(mat.dims[0]) ])

def Det(mat):
    if mat.dims[0] == 2:
        return mat[0,0]*mat[1,1]-mat[0,1]*mat[1,0]
    elif mat.dims[0] == 1:
        return mat[0]

def Pow(a, b):
    return exp (log(a)*b)
    
def NeoHook (C):
    return 0.5 * mu * (Trace(C-I) + 2*mu/lam * Pow(Det(C), -lam/2/mu) - 1)



In [7]:
I = IdentityCF(mesh.dim)
F = I + u.Deriv()   # attention: row .. component, col .. derivative
C = F * F.trans  

factor = Parameter(1.0)
a = BilinearForm(V, symmetric=False)
a += SymbolicEnergy(  (-factor * force * u ).Compile() )
a += SymbolicEnergy(  NeoHook (C).Compile() )

We want to solve the minimization problem for $\gamma=5$. Due to the high nonlinearity in the problem, the Newton iteration will not convergence with any initial guess. We approach the case $\gamma=5$ by solving problems with $\gamma=i/10$ for $i=1,\dots,50$ and taking the solution of the previous problem as an initial guess.

In [8]:
gfu = GridFunction(V)
gfu.vec[:] = 0

Draw (gfu, mesh, "u")
SetVisualization (deformation=True)

res = gfu.vec.CreateVector()
du = gfu.vec.CreateVector()

for loadstep in range(50):
    print ("loadstep", loadstep)
    factor.Set ((loadstep+1)/10)
    SolveNonlinearMinProblem(a,gfu)
    Redraw()

loadstep 0
Newton iteration 0
energy =  87.49999999999976
<A u 0 , A u 0 >_{-1}^0.5 =  0.0037796447300922735
Newton iteration 1
energy =  87.49999285841852
<A u 1 , A u 1 >_{-1}^0.5 =  1.0457509820941419e-06
Newton iteration 2
energy =  87.49999285841795
<A u 2 , A u 2 >_{-1}^0.5 =  1.001347511454856e-13
Newton iteration 3
energy =  87.49999285841795
<A u 3 , A u 3 >_{-1}^0.5 =  8.458275843570262e-16
loadstep 1
Newton iteration 0
energy =  87.49997857652963
<A u 0 , A u 0 >_{-1}^0.5 =  0.003778632421142671
Newton iteration 1
energy =  87.49997143877349
<A u 1 , A u 1 >_{-1}^0.5 =  1.0455638500319132e-06
Newton iteration 2
energy =  87.49997143877295
<A u 2 , A u 2 >_{-1}^0.5 =  9.960870024879529e-14
loadstep 2
Newton iteration 0
energy =  87.49994288264581
<A u 0 , A u 0 >_{-1}^0.5 =  0.0037776203042638964
Newton iteration 1
energy =  87.49993574871297
<A u 1 , A u 1 >_{-1}^0.5 =  1.0453762279700383e-06
Newton iteration 2
energy =  87.49993574871243
<A u 2 , A u 2 >_{-1}^0.5 =  9.95472

## Allen-Cahn equation

The Allen-Cahn equations describe the process of phase separation and is the ($L_2$) gradient-flow equation to the energy
$$E(v)=\int_\Omega \epsilon |\nabla v|^2 + v^2(1-v^2) dx$$

i.e. the solution to the Allen-Cahn equation solves
$$\partial_t u=\delta E \delta u$$
The quantity $u$ is an indicator for a phase where $−1$ refers to one phase and $1$ to another phase. The equation has two driving forces:

- $u$ is pulled into one of the two minima $(−1 \text{ and } 1)$ of the nonlinear term $u^2(1−u^2)$ (separation of the phases)
- the diffusion term scaled with \epsilon enforces a smooth transition between the two phases.
- $\epsilon$ determines the size of the transition layer

We use the "SymbolicEnergy" feature to formulate the energy minimization problem and combine it with an implicit Euler discretization:
$$Mu^{n+1}−Mu^n=\Delta t\underbrace{\frac{\delta E}{\delta u}}_{=:A(u)} (u^{n+1})$$
which we can interpreted as a nonlinear minimization problem again with the energy
$$E^{IE}(v)=\int_\Omega \frac{\epsilon}{2} + v^2(1-v^2) + \frac{1}{2 \Delta t} |v−u^n|^2 dx$$

To solve the nonlinear equation at every time step we again rely on Newton's method.

In [9]:
mesh = Mesh(uniform_1d_mesh(periodic=True))
V = Periodic(H1(mesh, order=4, dirichlet=[]))

u = V.TrialFunction()

eps = 4e-3
dt = 1e-1

gfu = GridFunction(V)
gfuold = GridFunction(V)

a = BilinearForm (V, symmetric=False)
a += SymbolicEnergy (eps/2*grad(u)*grad(u)
                     + ((1-u*u)*(1-u*u))
                     + 0.5/dt*(u-gfuold)*(u-gfuold))

In [10]:
from math import pi
gfu = GridFunction(V)
gfu.Set(sin(2*pi*x))
#gfu.Set(sin(1e8*x)) #<- essentially a random function
Draw(gfu,mesh,"u")
SetVisualization (deformation=False)
t = 0

In [11]:
for timestep in range(50):
    gfuold.vec.data = gfu.vec
    SolveNonlinearMinProblem(a,gfu)
    Redraw() 
    t += dt
    print("t = ", t)

Newton iteration 0
energy =  0.41447868713369207
<A u 0 , A u 0 >_{-1}^0.5 =  0.2810039875406398
Newton iteration 1
energy =  0.37685687746742225
<A u 1 , A u 1 >_{-1}^0.5 =  0.019232500760609793
Newton iteration 2
energy =  0.3766713122088701
<A u 2 , A u 2 >_{-1}^0.5 =  9.810674934673649e-05
Newton iteration 3
energy =  0.3766713073963163
<A u 3 , A u 3 >_{-1}^0.5 =  2.6608479350724507e-09
Newton iteration 4
energy =  0.3766713073963164
<A u 4 , A u 4 >_{-1}^0.5 =  1.908860517023638e-16
t =  0.1
Newton iteration 0
energy =  0.34379149393526287
<A u 0 , A u 0 >_{-1}^0.5 =  0.2363864831049363
Newton iteration 1
energy =  0.3169457069786601
<A u 1 , A u 1 >_{-1}^0.5 =  0.013548141208411147
Newton iteration 2
energy =  0.31685371604959267
<A u 2 , A u 2 >_{-1}^0.5 =  4.8142656231895404e-05
Newton iteration 3
energy =  0.3168537148907249
<A u 3 , A u 3 >_{-1}^0.5 =  6.328084519710255e-10
Newton iteration 4
energy =  0.3168537148907249
<A u 4 , A u 4 >_{-1}^0.5 =  2.3566270091051916e-16
t 